In [1]:
import nibabel as nib
from skimage.measure import marching_cubes
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def pad_edge_list(edges):
    padding = np.ones(edges.shape[0], int)*3
    edges_w_padding = np.vstack((padding, edges.T)).T
    return edges_w_padding

In [3]:
path = "C:/Users/aorhu/Masaüstü/body_mask.nii.gz"

In [4]:
body_segment = nib.load(path)
body_segment_data = body_segment.get_fdata()

In [5]:
verts, faces, norms, vals = marching_cubes(body_segment_data, level=0, step_size=1)

In [6]:
verts = verts/np.array(body_segment_data.shape) 

In [7]:
len(faces)

545964

In [8]:
len(verts)

273145

### open3d quadric_decimation

In [9]:
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))


decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, 10000)

In [10]:
decimated_mesh.triangles

std::vector<Eigen::Vector3i> with 9999 elements.
Use numpy.asarray() to access data.

In [11]:
np.asarray(decimated_mesh.vertices)[:,1]= np.asarray(decimated_mesh.vertices)[:,1]+1

In [12]:
original_mesh_paint = np.asarray([139,0,0])/255.0
decimated_mesh_paint = np.asarray([51,255,153])/255.0
mesh.paint_uniform_color(original_mesh_paint)
decimated_mesh.paint_uniform_color(decimated_mesh_paint)

TriangleMesh with 5873 points and 9999 triangles.

In [13]:
mesh.compute_vertex_normals()
decimated_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([mesh,decimated_mesh],mesh_show_back_face=True,mesh_show_wireframe=False)